In [33]:
import os
import getpass
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import EvaluationDataset, evaluate
from ragas.run_config import RunConfig
from ragas.metrics import faithfulness, answer_relevancy, answer_correctness, context_precision, context_recall
from datasets import Dataset
import time
import itertools
import pandas as pd

In [2]:
local_path = "../pdf/BILLS-119hr1eh.pdf"

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

In [3]:
len(data[0].page_content)

1136279

In [4]:
#Split and chunk the data
chunk_size = 750
chunk_overlap = 200

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

# Add the chunks to vector database, which uses nomic for model embeddings
vector_db = Chroma.from_documents(
                                    documents=chunks, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text"),
                                    collection_name="local-rag"
                                )

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [6]:
local_llm = "llama3.2"
llm = ChatOllama(model=local_llm)

#evaluator_llamma= FaithfulnessEvaluator(llm=llm)


# Set up a basic PromptTemplate as the backbones of the solution
# Ask the system to gather several responses and to limit response to 200 words
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
        template="""You are an AI Language model assistant. Your task is to generate three different versions of the given user question 
        to retrieve relevant documents from a vector database. Please be as concise as possible and limit your response to 200 words or less. 
        Original question: {question} """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

In [7]:
# use a ChatPromptTemplate to initiate a conversation, allowing the System to assume a Role
chat_template = """Answer the question based only on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(chat_template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

### Context-Driven responses

In [8]:
q_list = ['Resulting from this Act, which groups would be harmed most?',\
          'Resulting from this Act, which groups would benefit most?']

c_list = ['Role: you are a staunch Democrat',\
          'Role: you are a staunch Republican',\
          'Role: you are a US citizen with no political affiliation',\
          'Role: you are a wealthy investor with interests in oil, gas, and mining',\
          'Role: you are a person that enjoys recreating in public lands while living in Colorado']

In [9]:
def response_gen(q, context):
    """
    Invoke the language model chain with a given question and context, 
    then print the question, context, and the model's response.

    Args:
        q (str): The question to ask.
        context (str): The context or role to provide to the model.
    """
    response = chain.invoke(input={'context': context, 'question': q})
    
    print('*** \n')
    print(f"Question - {q}")
    print(f"Context - {context} \n")
    print("Response: \n", response)

    return response


def iterate_responses(q_list, c_list):

    response_list = []
    df_responses = []
    
    for combo in itertools.product(q_list, c_list):
        response = response_gen(q=combo[0], context=combo[1])

        data = {
        'question': [combo[0]],
        'context': [combo[1]],
        'response': [response]
        }
    
        df = pd.DataFrame(data)
    
        response_list.append(df)

    df_responses = pd.concat(response_list)
    df_responses.reset_index(inplace=True, drop=True)

    return df_responses

In [26]:
q_list1 = ['Resulting from this Act, which groups would be harmed most?']

c_comp_list = ['Role: you are a US citizen with no political affiliation',\
        'Role: you are a staunch Republican',\
        'Role: you are a staunch Republican, at a bar with only 15 seconds to explain your stance on the document',\
        'Role: you are a staunch Republican and Trump Supporter, at a bar with only 15 seconds to explain your stance on the document']

df_responses = iterate_responses(q_list=q_list1, c_list=c_comp_list)

*** 

Question - Resulting from this Act, which groups would be harmed most?
Context - Role: you are a US citizen with no political affiliation 

Response: 
 The question seems to be referring to the potential effects of the proposed legislation on various groups. Based on the provided text, it appears that the legislation may harm certain groups in the following ways:

1. Foreign nationals: The "unfair foreign tax" provision may affect foreign nationals who are subject to taxation by a foreign government. However, this is not necessarily a group that would be harmed most.
2. Small businesses: Some provisions, such as the repeal of EPA rules and NHTSA standards, may benefit small businesses by reducing regulations and costs. However, this is not necessarily a group that would be harmed most.
3. Low-income individuals: The legislation includes provisions related to health insurance, such as the "exchange enrollment verification requirement" and the "premium adjustment percentage." These

In [27]:
pd.set_option('display.max_colwidth', None)
df_responses.head()

,question,context,response
0,"Resulting from this Act, which groups would be harmed most?",Role: you are a US citizen with no political affiliation,"The question seems to be referring to the potential effects of the proposed legislation on various groups. Based on the provided text, it appears that the legislation may harm certain groups in the following ways:\n\n1. Foreign nationals: The ""unfair foreign tax"" provision may affect foreign nationals who are subject to taxation by a foreign government. However, this is not necessarily a group that would be harmed most.\n2. Small businesses: Some provisions, such as the repeal of EPA rules and NHTSA standards, may benefit small businesses by reducing regulations and costs. However, this is not necessarily a group that would be harmed most.\n3. Low-income individuals: The legislation includes provisions related to health insurance, such as the ""exchange enrollment verification requirement"" and the ""premium adjustment percentage."" These provisions may affect low-income individuals who rely on government-subsidized health insurance plans.\n\nConsidering these points, it's difficult to determine which group would be harmed most without more information. However, based on the context of the legislation, it appears that low-income individuals may be disproportionately affected by certain provisions.\n\nTo provide a more specific answer, I would need more information about the potential impacts of the legislation on different groups."
1,"Resulting from this Act, which groups would be harmed most?",Role: you are a staunch Republican,"The resulting groups that would be harmed most by this Act are low-income households and certain rural communities. The Act includes provisions aimed at reducing the cost of healthcare for some individuals and families, but these measures may disproportionately affect already vulnerable populations.\n\nSpecifically, Section 112103 of the Act limits Medicare coverage for certain individuals who are not lawfully present in the United States. This could harm low-income households that rely on government-subsidized health insurance.\n\nAdditionally, the Act includes provisions related to the Supplemental Nutrition Assistance Program (SNAP) and the Low-Income Home Energy Assistance Program (LIHEAP). These programs provide critical support for low-income families struggling with food insecurity and energy costs. Changes to these programs could exacerbate existing inequalities and harm already vulnerable populations.\n\nFurthermore, the Act includes provisions that may negatively impact rural communities. For example, Section 44105 of the Act modifies Medicaid provider screening requirements, which could lead to reduced access to healthcare services for rural residents who are already underserved.\n\nOverall, while this Act may have some benefits, its provisions could result in significant harm to low-income households and certain rural communities."
2,"Resulting from this Act, which groups would be harmed most?","Role: you are a staunch Republican, at a bar with only 15 seconds to explain your stance on the document","""This bill targets and harms foreign-influenced entities, private foundations in discriminatory foreign countries, and certain corporations owned by those nations. It's a direct attack on our sovereignty and values."" (15-second response)"
3,"Resulting from this Act, which groups would be harmed most?","Role: you are a staunch Republican and Trump Supporter, at a bar with only 15 seconds to explain your stance on the document","""Folks, let me tell you, this Act is a disaster. It's a total disaster. The groups that are gonna be harmed the most are the American workers, especially those in the manufacturing sector who are already struggling to stay competitive with foreign labor. And let's not forget our allies abroad, they're gonna feel the pinch too. And of course, our fellow Trump supporters, we're gonna have to take on the deep state and the

### Leverage RAGAS for Evaluating Prompts

In [32]:
# load_dotenv()

# if not os.environ.get("OPENAI_API_KEY"):
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# llm_eval = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Build query engine
# def build_query_engine(llm):
#     loader = TextLoader("./notebooks_nyc_wikipedia_nyc_text.txt")
#     data = loader.load()

#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
#     all_splits = text_splitter.split_documents(data)

#     vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

#     return RetrievalQA.from_chain_type(
#         llm,
#         retriever=vectorstore.as_retriever(),
#         return_source_documents=True
#     )

In [46]:
# Evaluation questions, contexts, and answers
eval_questions = ["Resulting from this Act, which groups would be harmed most?"]

eval_answers = [
   "The people who would be harmed most by this Act are the non-citizen individuals and foreign corporations. This Act seems to target those who own more than a certain percentage of stock in these entities or have significant ownership interests. It's essentially aimed at preventing fraud, waste, and abuse, but it also has implications for international business dealings and non-citizen residents."
]

contexts = [
    "Role: you are a staunch Republican, at a bar with only 15 seconds to explain your stance on the document"
]

# pull ground truth from answer with context = 'US citizen with no political affiliation'
ground_truth = [
   "Based on the provided text, it appears that the legislation may harm certain groups in the following ways: Foreign nationals: The 'unfair foreign tax' provision may affect foreign nationals who are subject to taxation by a foreign government. However, this is not necessarily a group that would be harmed most. Small businesses: Some provisions, such as the repeal of EPA rules and NHTSA standards, may benefit small businesses by reducing regulations and costs. However, this is not necessarily a group that would be harmed most. Low-income individuals: The legislation includes provisions related to health insurance, such as the 'exchange enrollment verification requirement' and the 'premium adjustment percentage.' These provisions may affect low-income individuals who rely on government-subsidized health insurance plans." 
]

examples = [
    {"query": q, "ground_truth": [eval_answers[i]], "contexts": [contexts[i]]}
    for i, q in enumerate(eval_questions)
]

examples

[{'query': 'Resulting from this Act, which groups would be harmed most?',
  'ground_truth': ["The people who would be harmed most by this Act are the non-citizen individuals and foreign corporations. This Act seems to target those who own more than a certain percentage of stock in these entities or have significant ownership interests. It's essentially aimed at preventing fraud, waste, and abuse, but it also has implications for international business dealings and non-citizen residents."],
  'contexts': ['Role: you are a staunch Republican, at a bar with only 15 seconds to explain your stance on the document']}]

In [29]:
# Run the queries as a batch for efficiency
predictions = chain.batch([ex["query"] for ex in examples])

In [56]:
dataset_dict = {
    "user_input": eval_questions[0],
    "response": eval_answers[0],
    "retrieved_contexts": contexts,
    "reference": ground_truth[0]
}

dataset_dict


{'user_input': 'Resulting from this Act, which groups would be harmed most?',
 'response': "The people who would be harmed most by this Act are the non-citizen individuals and foreign corporations. This Act seems to target those who own more than a certain percentage of stock in these entities or have significant ownership interests. It's essentially aimed at preventing fraud, waste, and abuse, but it also has implications for international business dealings and non-citizen residents.",
 'retrieved_contexts': ['Role: you are a staunch Republican, at a bar with only 15 seconds to explain your stance on the document'],
 'reference': "Based on the provided text, it appears that the legislation may harm certain groups in the following ways: Foreign nationals: The 'unfair foreign tax' provision may affect foreign nationals who are subject to taxation by a foreign government. However, this is not necessarily a group that would be harmed most. Small businesses: Some provisions, such as the re

In [57]:
# EvaluationDataset standardizes the collected evaluation examples into a structured format
evaluation_dataset = EvaluationDataset.from_list([dataset_dict])
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [62]:
# Evaluate using Ragas
result = evaluate(
    evaluation_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        #context_recall,
    ],
    llm=llm,
    embeddings=OllamaEmbeddings(model="nomic-embed-text"),
    run_config =RunConfig(timeout=500, max_retries=3, max_wait=50, log_tenacity=False),
)

print(result)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Exception raised in Job[0]: OutputParserException(Invalid json output: The provided context was somewhat useful in arriving at the given answer. The context provides a role-playing scenario and mentions specific groups that may be affected by the legislation, such as foreign nationals, small businesses, and low-income individuals. However, it does not directly address the question of which group would be harmed most.

Given question, answer and context verify if the context was useful in arriving at the given answer. Give verdict as \
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE )
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt sta

{'context_precision': nan, 'faithfulness': nan, 'answer_relevancy': 0.9492}
